# Deploying Campaigns and Interacting with Them

At this point there are solutions and at least one version for each that has been created. Once they are deployed it is possible to get recommendations from them and a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns, and then once they are active there are resources for querying the recommendations and then helper functions to digest the output into something a bit more human readable. 

As you are working through examples with your customers you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

## Initial Setup

To get started, once again imports, loading previous values, and loading the SDK.

In [1]:
import boto3
from time import sleep
import subprocess
import pandas as pd
import json
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from datetime import datetime

In [9]:
%store -r

In [3]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## Creating Campaigns

A campaign is a hosted solution version, pricing is done by estimating throughput capacity (requests from users for personalization per second). This service like many within AWS will automatically scale based on demand but if latency is critical you may want to provision ahead to the larger demand. Given this is purely a POC and a demo, all capacity limits are set to 1. The code below will just create the campaigns. Again in previous notebooks you may have seen while loops that polled, given that we want to execute multiple deployments at the same time the loops have been removed. Progress will be checked in another tab via the console just as you did for the solution version creation. This time instead of clicking on `Solutions and recipes` click the `Campaigns` link to the right to see their progress.

#### HRNN

In [4]:
hrnn_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-hrnn",
    solutionVersionArn = hrnn_solution_version_arn,
    minProvisionedTPS = 1
)

hrnn_campaign_arn = hrnn_create_campaign_response['campaignArn']
print(json.dumps(hrnn_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-poc-hrnn",
  "ResponseMetadata": {
    "RequestId": "5720e15d-7e7e-4594-87e6-f319375324ce",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 13 Jan 2020 01:03:11 GMT",
      "x-amzn-requestid": "5720e15d-7e7e-4594-87e6-f319375324ce",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### SIMS

In [5]:
sims_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-SIMS",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = sims_create_campaign_response['campaignArn']
print(json.dumps(sims_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-poc-SIMS",
  "ResponseMetadata": {
    "RequestId": "f51eb3bc-f322-4c11-9f97-38340aaee0fe",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 13 Jan 2020 01:03:43 GMT",
      "x-amzn-requestid": "f51eb3bc-f322-4c11-9f97-38340aaee0fe",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Personalized Ranking

In [6]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "97e00eae-6790-4b19-aada-2f547ee45cf6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 13 Jan 2020 01:04:17 GMT",
      "x-amzn-requestid": "97e00eae-6790-4b19-aada-2f547ee45cf6",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


This process should take no more than 15 minutes to complete for all your campaigns.

## Interacting with Campaigns

Now that they are all deployed and active we can start to get recommendations via the API call. Each of these behave in slightly different ways as they serve a different use case.  The order will be switched up a bit to deal with the possible complexities in ascending order(simplest first).

That said you may need a few supporting functions to help make sense of the results from the service. Personalize returns only an `item_id`. This is great for keeping data compact but it means you need to query the real DB or some lookup table to get a human readable result for the notebooks. The first few cells are going to create that for this particular example. 

In [13]:
# Create a dataframe for the items by reading in the correct source CSV.
items_df = pd.read_csv(data_dir + '/artists.dat', delimiter='\t', index_col=0)
# Render some sample data
items_df.head(5)

,name,url,pictureURL
id,,,
1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


By defining the ID column as the index column it is trivial to return an artist by just doing this:

In [16]:
item_id_example = 987
artist = items_df.loc[item_id_example]['name']
print(artist)

Earth, Wind & Fire


That isn't terrible but would get messy to repeat everywhere in our code so the function below will clean that up.

In [17]:
def get_artist_by_id(artist_id, artist_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case of anything going wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return artist_df.loc[int(artist_id)]['name']
    except:
        return "Error obtaining artist"

To test that out, a few simple values and to see what happens with errors:

In [19]:
# A known good id
print(get_artist_by_id(artist_id="987"))
# A bad type of value
print(get_artist_by_id(artist_id="987.9393939"))
# Really bad values
print(get_artist_by_id(artist_id="Steve"))

Earth, Wind & Fire
Error obtaining artist
Error obtaining artist


Great now we have a way of rendering results, now we'd like to select 3 random artists from our dataframe and determine their SIMS results. 

In [21]:
samples = items_df.sample(3)
samples

,name,url,pictureURL
id,,,
3062,Emmanuel,http://www.last.fm/music/Emmanuel,http://userserve-ak.last.fm/serve/252/51244263...
17251,Smoke or Fire,http://www.last.fm/music/Smoke+or+Fire,http://userserve-ak.last.fm/serve/252/51792215...
11734,nosnow/noalps,http://www.last.fm/music/nosnow%252Fnoalps,http://userserve-ak.last.fm/serve/252/61748427...


Now go forth and get some recommendations for just the first known item ( Earth Wind and Fire )

In [22]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(987),
)

In [23]:
item_list = get_recommendations_response['itemList']

In [24]:
for item in item_list:
    print(get_artist_by_id(artist_id=item['itemId']))

The Byrds
Johnny Cash
Lacrimas Profundere
Neil Young
Jethro Tull
Bob Dylan
Amorphis
George Harrison
Motörhead
Bruce Springsteen
John Lennon
The Who
The Rolling Stones


This is an OK list but it would be really cool to see how the collection of artists render in a nice Dataframe, the code below will do just that.

In [40]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, artist_ID):
    # Get the artist name
    artist_name = get_artist_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(artist_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_artist_by_id(item['itemId'])
        recommendation_list.append(artist)
    print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [artist_name])
    # Add this dataframe to the old one
    recommendations_df = recommendations_df.join(new_rec_DF)
    return recommendations_df

recommendations_df = pd.DataFrame()

artists = samples.index.tolist()

print(artists)

for artist in artists:
    print(artist)
    recommendations_df = get_new_recommendations_df(recommendations_df, artist)

recommendations_df

[3062, 17251, 11734]
3062
['Britney Spears', 'Depeche Mode', 'Lady Gaga', 'Madonna', 'Christina Aguilera', 'Muse', 'The Beatles', 'Rihanna', 'Radiohead', 'Coldplay', 'Katy Perry', 'Michael Jackson', 'Kylie Minogue', 'Avril Lavigne', 'Paramore', 'Placebo', 'Beyoncé', 'Pink Floyd', 'The Killers', 'Metallica', 'Björk', 'Mariah Carey', 'Arctic Monkeys', 'Led Zeppelin', 'Queen']
17251
['Britney Spears', 'Depeche Mode', 'Lady Gaga', 'Madonna', 'Christina Aguilera', 'Muse', 'The Beatles', 'Rihanna', 'Radiohead', 'Coldplay', 'Katy Perry', 'Michael Jackson', 'Kylie Minogue', 'Avril Lavigne', 'Paramore', 'Placebo', 'Beyoncé', 'Pink Floyd', 'The Killers', 'Metallica', 'Björk', 'Mariah Carey', 'Arctic Monkeys', 'Led Zeppelin', 'Queen']
11734
['Britney Spears', 'Depeche Mode', 'Lady Gaga', 'Madonna', 'Christina Aguilera', 'Muse', 'The Beatles', 'Rihanna', 'Radiohead', 'Coldplay', 'Katy Perry', 'Michael Jackson', 'Kylie Minogue', 'Avril Lavigne', 'Paramore', 'Placebo', 'Beyoncé', 'Pink Floyd', 'The 

,Emmanuel,Smoke or Fire,nosnow/noalps
